## Running prompts on OpenAI

In [1]:
%pip install markdown
%pip install annoy
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI # OpenAI API
import json
import requests # to download some resources
import os # file operations
import numpy as np # linear algebra
import pandas as pd # data processing
from markdown import markdown # to render markdown
from IPython.display import Markdown
import annoy # Approximate Nearest Neighbors Oh Yeah for fast searching
import pickle
from annoy import AnnoyIndex

/Applications/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# read in our api key
with open('../api-keys/our_api_key.txt', 'r') as file:
    api_key = file.read().replace('\n', '')

# read in finns api key (the one we'll use for testing)
with open('../api-keys/finns_api_key.txt', 'r') as file:
    finns_api_key = file.read().replace('\n', '')

In [4]:
# Setting the OpenAI
client = OpenAI(api_key=finns_api_key) # using finns for testing

#### Sample of countries

In [5]:
# Sample of countries
sample_countries = ['Finland','Netherlands','Denmark','Norway', 'Chile']

# Folder with raw PDFs
folder_mds = "../data/3-naps-md"

# Importing MDs files
for file in os.listdir(folder_mds):
    for country in sample_countries:
        if country in file:  
            with open(os.path.join(folder_mds, file), "r", encoding="utf-8") as md_file:
                content = md_file.read()
            globals()[country.lower()] = content # Saving the MD file in lowercase

### 1. Using the WHOLE document - CHILE

In [12]:
country_test = chile

#### **Question: Period for NAP**

In [13]:
response_period = client.chat.completions.create(
    model="gpt-4o-mini",
    response_format={"type":"json_object"},
    messages=[
        {"role": "system", "content": """
         You are an assistant that extract information from national action plans for antimicrobial resistance. Be concise and rely only on the text content.

         From this National Action Plan, extract the period considered for the National Action Plan and return it in a structured JSON format.
         The response **must** follow this exact JSON structure:
            {
                "country": "Name of country as string",
                "period_start": "The year when starts the period as numeric",
                "period_end": "The year when ends the period as numeric"
            }
         """},
        {"role": "user", "content": country_test},
    ]
)
# Print the JSON response
print(response_period.choices[0].message.content)

{
    "country": "Chile",
    "period_start": 2021,
    "period_end": 2025
}


In [14]:
# see how many tokens that last query used
response.usage

CompletionUsage(completion_tokens=28, prompt_tokens=22457, total_tokens=22485, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

#### **Question: About policies/objectives/goals/strategies**

In [46]:
response_goals = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature= 0.2,
    top_p = 0.5,
    response_format={"type":"json_object"},
    messages=[
        {"role": "system", "content": """

         You are an assistant that extract information from the national action plan for antimicrobial resistance. Be concise and extract the text content literally, without any wording or paraphrase. 
         
         Your task is to identify every actionable policy goal in the document and write it down in the original language in JSON format. 
         
         For each goal, you must put your findings in this JSON structure. Try to extract the maximum number of policies as you can. If any of the fields are not applicable, mark them as null:

            policies = [
            {
                "goal": the policy goal set out,
                "metric": the metric used to measure the policy goal,
                "steps": a list specific steps or specific policies set out to achieve the policy goal,
                "agent": a list of agents responsible for the policy goal,
                "time_limit" : the time limit set for the policy goal beyond the general time limit of the national action plan,
                "funding": any funding allocated to the achievement policy goal and how it is denoted,
            }
         ]

         """},
        {"role": "user", "content": country_test}
    ]
)
# Print the JSON response
print(response_goals.choices[0].message.content)

{
  "policies": [
    {
      "goal": "Fortalecer la concientización y la formación respecto a la RAM en la comunidad en general y en grupos específicos.",
      "metric": null,
      "steps": [
        "Desarrollar e implementar estrategias comunicacionales educativas dirigidas a la comunidad.",
        "Promover en las y los docentes la incorporación de las temáticas de resistencia y uso prudente y responsable de los antimicrobianos en ámbitos educacionales.",
        "Generar instancias para la creación, actualización y perfeccionamiento de conocimientos y competencias sobre RAM en prescriptores y otros profesionales.",
        "Generar instancias de capacitación locales para profesionales de la salud en relación con el control de infecciones, el uso de antimicrobianos y la RAM."
      ],
      "agent": [
        "Ministerio de Salud",
        "Ministerio de Educación"
      ],
      "time_limit": null,
      "funding": null
    },
    {
      "goal": "Fortalecer el sistema nacional

In [14]:
# see how many tokens that last query used
response_goals.usage

CompletionUsage(completion_tokens=1255, prompt_tokens=15948, total_tokens=17203, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [106]:
data_goals = json.loads(response_goals.choices[0].message.content)
policies = data_goals["policies"]
df_policies = pd.DataFrame(policies)
df_policies["steps"] = df_policies["steps"].apply(lambda x: "\n".join(x))
df_policies["agent"] = df_policies["agent"].apply(lambda x: ", ".join(x))


In [107]:
df_policies

,goal,metric,steps,agent,time_limit,funding
0,Fortalecer la concientización y la formación r...,None,Desarrollar e implementar estrategias comunica...,"Ministerio de Salud, Ministerio de Educación",None,None
1,Fortalecer el sistema nacional de vigilancia d...,None,Desarrollar un sistema de vigilancia integrada...,"Ministerio de Salud, Instituto de Salud Pública",None,None
2,Prevenir y controlar las infecciones asociadas...,None,Prevenir las Infecciones Asociadas a la Atenci...,"Ministerio de Salud, Ministerio de Agricultura",None,None
3,Regular y vigilar el uso de los antimicrobianos.,None,Regular el uso de antimicrobianos en las perso...,"Ministerio de Salud, Ministerio de Agricultura",None,None
4,Mejorar el acceso a la información y fomentar ...,None,Mejorar el acceso a la información sobre RAM.\...,"Ministerio de Ciencia, Tecnología, Conocimient...",None,None


In [62]:
json_goals = json.loads(response_goals.choices[0].message.content)

all_goals = []
all_steps = []

for policy in json_goals["policies"]:
    all_goals.append(policy["goal"])  
    all_steps.extend(policy["steps"])
    
all_goals_str = json.dumps(all_goals, ensure_ascii=False)
all_steps_str = json.dumps(all_steps, ensure_ascii=False)    

#### **Question: Classification of polices by sector**

In [110]:
response_sectors = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature= 0.2,
    top_p = 0.5,
    response_format={"type":"json_object"},
    messages=[
        {"role": "system", "content": f"""
         
        You are an assistant that classify information from national action plans for antimicrobial resistance. Be concise and rely only on the text content.

        Your task is to classify every actionable policy goal into one or more of the following priority sectors defined by the World Health Organization:
        - Human Health
        - Animal Health
        - Environment and Agriculture.
        - General/All

        If a policy goal does not fit into any of these categories, classify it as `null`, and if the scope of policy goal is general or include all sectors, mark it as `General/All`. 
        
        The list of policy goals are in the following lists: {all_goals_str} and {all_steps_str}. 
        
        Use all the policy goals provided in the lists and find the explanation of policy goals that are in national action plan to help you to classify them properly into the priority sectors.
        
        Do not forget to include **ALL** goals of the lists.

        Return the results in this JSON format:
        {{
            "sectors": [
                {{
                    "sector": "General/All",
                    "policies": [...],
                    "number_of_policies": number of policies
                }},         
                {{
                    "sector": "Human Health",
                    "policies": [...],
                    "number_of_policies": number of policies
                }},
                {{
                    "sector": "Animal Health",
                    "policies": [...],
                    "number_of_policies": number of policies
                }},
                {{
                    "sector": "Environment and Agriculture",
                    "policies": [...],
                    "number_of_policies": number of policies
                }},
                {{
                    "sector": null,
                    "policies": [...],
                    "number_of_policies": number of policies
                }}
            ]
        }}
        
        """},
        {"role": "user", "content": country_test}
    ]
)

# Print the JSON response
print(response_sectors.choices[0].message.content)

{
    "sectors": [
        {
            "sector": "General/All",
            "policies": [
                "Fortalecer la concientización y la formación respecto a la RAM en la comunidad en general y en grupos específicos.",
                "Mejorar el acceso a la información y fomentar la investigación relacionada con la RAM."
            ],
            "number_of_policies": 2
        },
        {
            "sector": "Human Health",
            "policies": [
                "Fortalecer el sistema nacional de vigilancia de la resistencia a los antimicrobianos.",
                "Prevenir y controlar las infecciones asociadas a la salud humana y a la sanidad vegetal y animal.",
                "Regular y vigilar el uso de los antimicrobianos.",
                "Desarrollar un sistema de vigilancia integrada sobre RAM en microorganismos seleccionados.",
                "Fortalecer el sistema de monitoreo de Agentes con Resistencia a los Antimicrobianos de Importancia en Salud Pública.

In [113]:
data_sectors = json.loads(response_sectors.choices[0].message.content)
sectors = data_sectors["sectors"]
df_sectors = pd.DataFrame(sectors)
df_sectors = df_sectors.explode("policies", ignore_index=True)
df_sectors 

In [114]:
df_sectors

,sector,policies,number_of_policies
0,General/All,Fortalecer la concientización y la formación r...,2
1,General/All,Mejorar el acceso a la información y fomentar ...,2
2,Human Health,Fortalecer el sistema nacional de vigilancia d...,15
3,Human Health,Prevenir y controlar las infecciones asociadas...,15
4,Human Health,Regular y vigilar el uso de los antimicrobianos.,15
5,Human Health,Desarrollar un sistema de vigilancia integrada...,15
6,Human Health,Fortalecer el sistema de monitoreo de Agentes ...,15
7,Human Health,Fortalecer la vigilancia de la tuberculosis re...,15
8,Human Health,Fortalecer la vigilancia de la resistencia en ...,15
9,Human Health,Prevenir las Infecciones Asociadas a la Atenci...,15


#### **Question: Classification of polices by strategic objectives**

In [69]:
response_strategies = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature= 0.2,
    top_p = 0.5,
    response_format={"type":"json_object"},
    messages=[
        {"role": "system", "content": f"""
         
        You are an assistant that classify information from national action plans for antimicrobial resistance. Be concise and rely only on the text content.

        Your task is to classify every actionable policy goal into one or more of the following broad strategic objectives defined by the World Health Organization:
         
        - Objective 1: Improve awareness and understanding of antimicrobial resistance through effective communication, education and training.
        - Objective 2: Strengthen the knowledge and evidence base through surveillance and research.
        - Objective 3: Reduce the incidence of infection through effective sanitation, hygiene and infection prevention measures.
        - Objective 4: Optimize the use of antimicrobial medicines in human and animal health.
        - Objective 5: Develop the economic case for sustainable investment that takes account of the needs of all countries, and increase investment in new medicines, diagnostic tools, vaccines and other interventions

        If a policy goal does not fit into any of these categories, classify it as `null`, and if the scope of policy goal is general or include all sectors, mark it as `General/All`. 
        
        The list of policy goals are in the following lists: {all_goals_str} and {all_steps_str}. 
        
        Use all the policy goals provided in the lists and find the explanation of policy goals that are in national action plan to help you to classify them properly into the priority sectors.
        
        Do not forget to include **ALL** goals of the lists.

        Return the results in this JSON format:
        {{
            "sectors": [
                {{
                    "strategy": Name of the broad strategic objective,
                    "policies": [...],
                    "number_of_policies": number of policies
                }},
            ]
        }}
        
        """},
        {"role": "user", "content": country_test}
    ]
)

# Print the JSON response
print(response_strategies.choices[0].message.content)

{
    "sectors": [
        {
            "strategy": "Objective 1: Improve awareness and understanding of antimicrobial resistance through effective communication, education and training.",
            "policies": [
                "Fortalecer la concientización y la formación respecto a la RAM en la comunidad en general y en grupos específicos.",
                "Desarrollar e implementar estrategias comunicacionales educativas dirigidas a la comunidad.",
                "Promover en las y los docentes la incorporación de las temáticas de resistencia y uso prudente y responsable de los antimicrobianos en ámbitos educacionales.",
                "Generar instancias para la creación, actualización y perfeccionamiento de conocimientos y competencias sobre RAM en prescriptores y otros profesionales.",
                "Generar instancias de capacitación locales para profesionales de la salud en relación con el control de infecciones, el uso de antimicrobianos y la RAM."
            ],
    

#### **Question: Progress report**

In [97]:
response_progress = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature= 0.2,
    top_p = 0.5,
    response_format={"type":"json_object"},
    messages = [
        {"role": "system", "content": """

         You are an assistant that extract information from national action plans for antimicrobial resistance. Be concise and rely only on the text content.

         Your task is to identify regular progress reports or tracking reports planned for monitoring the whole implementation of the policy goals over the period considered.
        
         For every regular review or progress report, you must put your findings in this JSON structure. If there is not any regular progress report or tracking report, mark them as null:

            reports = [
            {
                "report": name of the regular review or progress report,
                "frequency": the frequency of the regular review or progress report,
                "agent": a list of agents responsible for the regular review or progress report,
            }
         ]
         """},
         {"role": "user", "content": country_test}
    ]
)
# Print the JSON response
print(response_progress.choices[0].message.content)

{
    "reports": [
        {
            "report": "Informe de situación 2020",
            "frequency": "Anual",
            "agent": ["Ministerio de Salud"]
        },
        {
            "report": "Reporte periódico de vigilancia de RAM",
            "frequency": "Periódico",
            "agent": ["Comité científico-técnico"]
        }
    ]
}
